In [ ]:
import re
import sys
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import IPython.display

import pydicom

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys

In [ ]:
percent_deviation = 2
mm_dist_threshold = 0.5

In [ ]:
data_paths = list(pathlib.Path(r'C:\Users\sbiggs\Desktop\20191223_logfile_compare').glob('*/*'))
data_paths

In [ ]:
def get_file_type(input_paths, file_type):
    paths = [path for path in input_paths if file_type in path.name]
    assert len(paths) == 1
    return paths[0]

In [ ]:
def plot_gamma_hist(gamma, percent, dist):
    valid_gamma = gamma[~np.isnan(gamma)]

    plt.hist(valid_gamma, 50, density=True)
    pass_ratio = np.sum(valid_gamma <= 1) / len(valid_gamma)

    plt.title("Local Gamma ({0}%/{1}mm) | Percent Pass: {2:.2f} % | Max Gamma: {3:.2f}".format(percent, dist, pass_ratio*100, np.max(valid_gamma)))

In [ ]:
def markdown_print(string):
    IPython.display.display(IPython.display.Markdown(string))

In [ ]:
def plot_and_save_results(mu_density_angles, mudensity_tel, mudensity_trf, gammas, output_dir):
    for gantry_angle, tel_result, trf_result, gamma in zip(mu_density_angles, mudensity_tel, mudensity_trf, gammas):
        markdown_print(f"### Gantry Angle: {gantry_angle}")

        plt.figure()
        pymedphys.mudensity.display(grid, tel_result)
        plt.title("Monaco Plan")
        plt.savefig(output_dir.joinpath(f"G{gantry_angle}_001_monaco.png"))

        plt.figure()
        pymedphys.mudensity.display(grid, tel_result)
        plt.title("Logfile Result")
        plt.savefig(output_dir.joinpath(f"G{gantry_angle}_002_logfile.png"))

        diff = trf_result - tel_result
        largest_item = np.max(np.abs(diff))

        plt.figure()
        pymedphys.mudensity.display(grid, diff, cmap="seismic", vmin=-largest_item, vmax=largest_item)
        plt.title("Logfile - Monaco")
        plt.savefig(output_dir.joinpath(f"G{gantry_angle}_003_diff.png"))

        plt.figure()
        pymedphys.mudensity.display(grid, gamma, cmap="coolwarm", vmin=0, vmax=2)
        plt.title(f"Local Gamma | {percent_deviation}%/{mm_dist_threshold}mm")
        plt.savefig(output_dir.joinpath(f"G{gantry_angle}_004_gamma_map.png"))

        plt.figure()
        plot_gamma_hist(gamma, percent_deviation, mm_dist_threshold)
        plt.savefig(output_dir.joinpath(f"G{gantry_angle}_005_gamma_hist.png"))

        plt.show()

In [ ]:
patient_ids = {path.parent.name for path in data_paths}
patient_ids

In [ ]:
grid = pymedphys.mudensity.grid()
coords = (grid['jaw'], grid['mlc'])

for patient_id in patient_ids:
    markdown_print(f"## Patient ID: {patient_id}")
    current_paths = [path for path in data_paths if path.parent.name == patient_id]
    
    tel_path = get_file_type(current_paths, "tel")
    trf_path = get_file_type(current_paths, "trf")
    
    results = trf_path.parent.joinpath("results")
    results.mkdir(exist_ok=True)
    
    delivery_tel = pymedphys.Delivery.from_monaco(tel_path)
    delivery_trf = pymedphys.Delivery.from_logfile(trf_path)
    
    mu_density_angles = set(delivery_tel.gantry)
    mudensity_tel = delivery_tel.mudensity(gantry_angles=mu_density_angles)
    mudensity_trf = delivery_trf.mudensity(gantry_angles=mu_density_angles)
    
    gammas = []
    for tel_result, trf_result in zip(mudensity_tel, mudensity_trf):
        gammas.append(
            pymedphys.gamma(coords, tel_result, coords, trf_result, percent_deviation, mm_dist_threshold, local_gamma=True))
        
    plot_and_save_results(mu_density_angles, mudensity_tel, mudensity_trf, gammas, results)